In [102]:
def infoGain(P,N):
    import math
    return -P/(P+N)*math.log2(P/(P+N))-N/(P+N)*math.log2(N/(P+N))

In [103]:
def getpath(nested_dict, value, prepath=0):
    for k, v in nested_dict.items():
        path = prepath
        if v == value: 
            return path+1
        elif hasattr(v, 'items'): 
            p = getpath(v, value, path) 
            if p is not None:
                return p

In [104]:
def finditem(obj, key):
    if key in obj: return key
    for k, v in obj.items():
        if isinstance(v,dict):
            item = _finditem(v, key)
            if item is not None:
                return item

In [114]:
def insertNode(tree,Node,AttPosNeg):

    for k, v in tree.items():
        if v == 'None':
            NV = {}
            for i in AttPosNeg:
                for j in AttPosNeg[i]:
                    NV[j]='None'
                    if AttPosNeg[i][j] == 0:
                        NV[j]='No'
                    if AttPosNeg[i][j] == 0:
                        NV[j]='Yes'
            tree[k]={Node:NV}
        elif hasattr(v, 'items'):
            p = insertNode(v, value)
            if p is not 'None':
                return p
    return tree

In [115]:
def countConcept(data,attr,val):
    from collections import Counter
    cnt=Counter(x for x in data[attr])
    return cnt[val]

In [116]:
def countAttr(data,attr,val,concept,cVal):
    count={}
    C = data[concept]
    A = data[attr]
    for a in range(len(data[attr])):
        for v in val:
            if v not in count:
                count[v] = 0
            if(A[a]==v and C[a]==cVal):
                if v not in count:
                    count[v]=1
                else:
                    count[v]=count[v]+1
    return count

In [ ]:
def getNextData(data,AttPosNeg,Node,tree):
    for nDi in AttPosNeg[Node]['Yes']:
        if finditem(tree, 'None')==nDi:
            iL = []
            iData = data[Node]
            for i in iData:
                if i == nDi:
                    pass
    pass #to be continued..

In [120]:
def getTree(data,AttributeList,concept,tree):
    flag = True
    Attr = {}
    Total=data.shape[0]
    for a in AttributeList:
        Attr[a] = list(set(data[a]))
        
    conceptVals = list(set(data[concept]))
    AttrCount = {}
    countC={}
    
    pathToNone = getpath(tree,'None')
#     print(pathToNone)
    if(pathToNone>0):
        for val in Attr:
            AttrCount[val] = {}
            for cVal in conceptVals:
                countC[cVal] = countConcept(data,concept,cVal)
            for cVal in conceptVals:
                AttrCount[val][cVal] = countAttr(data,val,Attr[val],concept,cVal)
        print(AttrCount)
        print(countC)
        EntropyAttr = {}
        AttPosNeg = {}
        ClassEntropy = infoGain(countC['Yes'],countC['No'])
        print("Class Entropy : ",ClassEntropy)
        for att in AttrCount:
            InfoGain = {}
            ap = {}
            an = {}
            for c in conceptVals:
                for val in AttrCount[att][c]:
                    if c == 'No':
                        an[val] = AttrCount[att][c][val]
                    if c == 'Yes':
                        ap[val] = AttrCount[att][c][val]
            AttPosNeg[att]={}
#             AttPosNeg[att][val] = {'Yes':ap,'No':an}
            TotalInfo = {}
            AttPosNeg[att] = {'Yes':ap,'No':an}
            for val in AttrCount[att][c]:
                TotalInfo[val] = ap[val]+an[val]
                if ap[val]==0 or an[val]==0:
                    InfoGain[val]=0
                else:
                    InfoGain[val] = infoGain(ap[val],an[val])
            print("Information Gain : ",InfoGain)
            for val in InfoGain:
                if att not in EntropyAttr:
                    EntropyAttr[att] = ( TotalInfo[val] / Total ) * InfoGain[val]
                else:
                    EntropyAttr[att] = EntropyAttr[att] + ( TotalInfo[val] / Total ) * InfoGain[val]
        print("Attribute Entropy : ",EntropyAttr)
        Gain = {}
        for g in EntropyAttr:
            if g not in Gain:
                Gain[g] = ClassEntropy - EntropyAttr[g]
        print("Gain : ",Gain)
        Node = max(Gain,key=Gain.get)
        print("Next root : ",Node)
        tree = insertNode(tree,Node,AttPosNeg[Node])
        print(tree)
        for d in range(pathToNone):
            newData = getNextData(data,AttPosNeg,Node,tree)
            tree = getTree(newData,AttributeList,concept,tree)
    else:
        return tree

In [121]:
def main():
    import pandas as pd
    from pandas import DataFrame 
    from collections import Counter
    data = DataFrame.from_csv('play.csv')
    AttributeList=list(data)[:-1]
    num_of_attributes=len(AttributeList)
    concept=str(list(data)[-1])
    tree = getTree(data,AttributeList,concept,{'root':'None'})

In [122]:
main()

{'Outlook': {'No': {'Sunny': 3, 'Overcast': 0, 'Rain': 2}, 'Yes': {'Sunny': 2, 'Overcast': 4, 'Rain': 3}}, 'Temperature': {'No': {'Cool': 1, 'Mild': 2, 'Hot': 2}, 'Yes': {'Cool': 3, 'Mild': 4, 'Hot': 2}}, 'Humidity': {'No': {'High': 4, 'Normal': 1}, 'Yes': {'High': 3, 'Normal': 6}}, 'Wind': {'No': {'Weak': 2, 'Strong': 3}, 'Yes': {'Weak': 6, 'Strong': 3}}}
{'No': 5, 'Yes': 9}
Class Entropy :  0.9402859586706311
Information Gain :  {'Sunny': 0.9709505944546686, 'Overcast': 0, 'Rain': 0.9709505944546686}
Information Gain :  {'Cool': 0.8112781244591328, 'Mild': 0.9182958340544896, 'Hot': 1.0}
Information Gain :  {'High': 0.9852281360342516, 'Normal': 0.5916727785823275}
Information Gain :  {'Weak': 0.8112781244591328, 'Strong': 1.0}
Attribute Entropy :  {'Outlook': 0.6935361388961918, 'Temperature': 0.9110633930116763, 'Humidity': 0.7884504573082896, 'Wind': 0.8921589282623617}
Gain :  {'Outlook': 0.24674981977443933, 'Temperature': 0.02922256565895487, 'Humidity': 0.15183550136234159, 'W